
**[PT]** Português

---

**[EN]** English



# Visão geral da base de dados

---

# Database overview 


## Setup

In [1]:
from timelink.api.database import TimelinkDatabase
from timelink import version
from ucalumni.config import default_db_url

print(f"Timelink API version: {version}")
print(f"Creating TimelinkDatabase instance from {default_db_url}")
db = TimelinkDatabase(db_url=default_db_url)


Timelink API version: 1.1.14
Creating TimelinkDatabase instance from sqlite:///../database/sqlite3/fauc3.db?check_same_thread=False


In [2]:
default_db_url

'sqlite:///../database/sqlite3/fauc3.db?check_same_thread=False'

## Database status

In [3]:
from sqlalchemy.sql import func
from timelink.api.models import Entity


with db.session() as session:
    classes = session.query(Entity.pom_class,func.count(Entity.pom_class)).group_by(Entity.pom_class).all()
    for c,n in classes:
        print(f'{n:8} | {c}')


     235 | act
 2840507 | attribute
      13 | class
  155320 | person
  205436 | relation
     235 | source


## Atributos inferidos

---

## Atributes inferred

In [4]:
from sqlalchemy import func
from sqlalchemy import select
from timelink.api.models import Attribute

attr_table = Attribute.__table__

stmt = select(attr_table.c.the_type,func.count().label('tot')).group_by('the_type')
print(stmt)
print()
attributes_extracted = []
with db.session() as session:
    # nml2 = session.query(Attribute.the_type,func.count().label('tot')).group_by(Attribute.the_type).all()
    nml = session.execute(stmt)
    for the_type, tot in nml:
        print(f'{tot:6} | {the_type}')
        attributes_extracted.append((the_type,tot))

attr_not_redundand = [(t,c) for (t,c) in attributes_extracted if "." not in t]

SELECT attributes.the_type, count(*) AS tot 
FROM attributes GROUP BY attributes.the_type

   357 | colegio
105297 | código-de-referência
105297 | data-do-registo
    13 | errata
 53745 | exame
104619 | faculdade
  7033 | faculdade-original
104619 | faculdade.ano
 87110 | grau
 87110 | grau.ano
 40673 | instituta
 40673 | instituta.ano
  2445 | matricula-classe
  2445 | matricula-classe.ano
   114 | matricula-classe.obrigado
   114 | matricula-classe.obrigado.ano
    33 | matricula-classe.ordinário
    33 | matricula-classe.ordinário.ano
     2 | matricula-classe.voluntário
     2 | matricula-classe.voluntário.ano
   170 | matricula-curso
   170 | matricula-curso.ano
313944 | matricula-faculdade
313944 | matricula-faculdade.ano
  9476 | matricula-faculdade.obrigado
  9476 | matricula-faculdade.obrigado.ano
  6513 | matricula-faculdade.ordinário
  6513 | matricula-faculdade.ordinário.an
  1546 | matricula-faculdade.voluntário
  1546 | matricula-faculdade.voluntário.a
   863 | matricula-

### Atributos inferidos (sem os atributos redundantes)

Os atributos redundantes são repetições 
de um atributos principal para facilitar 
determinadas pesquisas.

Todos os atributos com um "." no nome fornecem
uma variante do nome ou dovalor para facilitar pesquisas
e contagens. 

Por exemplo:

---

### Inferred attributes (without redundant attributes)

Redundant attributes are repetitions of an important
attribute with variants of the name or value to
facilitate retrieval or counts.

For instance:



| Atributo                     | Valor        | data 
|------------------------------|--------------|-----------
| faculdade                    | Cânones      | 1774-11-15
| faculdade.ano  	           | Cânones.1774 | 1774-11-15
| matricula-faculdade          | Matemática   |1775-00-00
| matricula-faculdade.obrigado | Matemática   | 1775-00-00



In [5]:
import pandas as pd

df = pd.DataFrame.from_records(attr_not_redundand, columns=['attr','n'])
df.sort_values('attr')

,attr,n
0,colegio,357
1,código-de-referência,105297
2,data-do-registo,105297
3,errata,13
4,exame,53745
5,faculdade,104619
6,faculdade-original,7033
7,grau,87110
8,instituta,40673
9,matricula-classe,2445


### Exemplo de registo complexo

---

### Example of complex record

Example of the current capabilities of the algorithm including correction of “Faculdade”, religious order, titles, enrollment at class level, exam results and degrees.

In [12]:
from timelink.pandas import group_attributes
from ucalumni.extractors import get_extractors
from ucalumni.aluno import get_and_process_aluno
from ucalumni.mapping import map_aluno_kperson

# id = '316297'
# id = '188758'
# id = '140543'
# id = "152674"  # com ordem religiosa
# id = '268660'  # com faculdade errada, deveria ser leis
# id = '152975'  # registo com errata local
# id = '223149'  # ficha remissiva
# par de fichas vide see-aka:
# id = "128053"
# id = "139670"
# id = '142381'  # André de Almada (com titulo)
# id = "215793"   # transition of plan
# id = '185916'  # com transição de plano e datas incompletas
# id = '203081'  # lente
# id = '232178'  # lente not title
# id = '143837'  # crash com "código"
# id = '129027'  # com "provas"
id = '140789'

exts = get_extractors()
print("Extractors currently active:")
[print("  ",ex.__name__) for ex in exts]
print()
aluno = get_and_process_aluno(id,db)
pass

Extractors currently active:
   extract_name_note_vid
   extract_colegio
   extract_titulo
   extract_ordem_religiosa
   extract_naturalidade
   extract_instituta
   extract_faculdade
   extract_graus
   extract_matriculas
   extract_exames
   extract_provas



### Show processing information

Copy the information to a markdown file it should render nicely

In [13]:

print("------------------------")
print(f"https://pesquisa.auc.uc.pt/details?id={id}")

print(aluno)
print("------------------------")



------------------------
https://pesquisa.auc.uc.pt/details?id=140789
# 140789 Gaspar de Abreu

## Original
```
Id: 140789
Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000521

Nome        : Gaspar de Abreu
Data inicial: 1574-11-01
Data final  : 1585-12-11
Filiação: Francisco Cardoso

Naturalidade: Fornos
Faculdade: Leis

Matrícula(s): 1577/10/11
1578/10/10
1579/10/09
1580/03/18
1585/10/04
Instituta: 
Bacharel: 1585/12/11
Formatura: 1585/12/11
Licenciado: 
Doutor:

Outras informações:
Provou cursar as lições de Prima e Véspera de Leis e as de Instituta: 01.11.1574 até 30.04.1575
1 curso de Artes no Código: 01.10.1575 até 31.05.1576
15.10.1576 até 15.04.1577
02.10.1577 até 15.05.1578
05.10.1578 até 15.06.1579

```
## Inferences:
* id:140789
* Nome: Gaspar de Abreu
* Data inicial:1574-11-01
* Data final: 1585-12-11
* Codigo de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000521

### Notes (sections and fields from the record):
|N   |proc   |section      |campo                 |data   

### Show in kleio format

In [8]:
kaluno = map_aluno_kperson(aluno)
print(kaluno.to_kleio())




n$Simão Cabral/m/id=129027/obs="""

      Id: 129027
      Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/C/000640

      Nome        : Simão Cabral
      Data inicial: 1540-10-25
      Data final  : 1551-07-02
      Filiação: Aires Pires Cabral
      Naturalidade: Tomar
      Faculdade: Dialética

      Matrícula(s): 25.10.1540

      Provou desde Outubro de 1548 até ao fim de 1550
      1 curso de Cânones em 1554
      3ª lição de Suficiência em Cânones, 07.03.1551
      Exame Privado - 27.06.1551
      Licenciado - 02.07.1551
  """
 rel$parentesco/filho/Aires Pires Cabral/129027-pai/data=1540-10-25
 atr$código-de-referência/"PT/AUC/ELU/UC-AUC/B/001-001/C/000640"/2024-11-21
 atr$data-do-registo/2024-11-21/2024-11-21
 atr$url/"http://pesquisa.auc.uc.pt/details?id=129027"/2024-11-21
 ls$uc-entrada/1540-10-25/1540-10-25
 ls$uc-saida/1551-07-02/1551-07-02
 ls$uc-entrada.ano/1540/1540-10-25
 ls$uc-saida.ano/1551/1551-07-02
 ls$nome/Simão Cabral/1540-10-25
 ls$nome-primeiro/Simão/1540-1

### Show canonic form

This is how the record should be registred

In [9]:
print(aluno.canonic())

https://pesquisa.auc.uc.pt/details?id=129027

# Id: 129027
# Simão Cabral

Codigo de referência
PT/AUC/ELU/UC-AUC/B/001-001/C/000640

Título
Simão Cabral

Datas de Produção
1540-10-25 a 1551-07-02

História administrativa/biográfica/familiar
Pai: Aires Pires Cabral

Naturalidade: Tomar


Âmbito e conteúdo

Faculdade: Cânones, 1540-10-25  (Ficha original: Dialética)
Faculdade: Dialética, 1540-10-25  (Ficha original: Dialética)
Faculdade nota: Aviso: faculdade corrigida
Faculdade nota: Faculdade corrigida de "Dialética" para "Cânones, Dialética". . 'Dialética' não é faculdade em 1551-07-02.  

Matrícula(s):
Cânones: 1540-10-25 (25.10.1540)
Dialética: 1540-10-25 (25.10.1540)

Exames:
1551-06-27:  Exame Privado,  (27.06.1551)

Provas:
desde Outubro de 1548 até ao fim de 1550:1548-00-00
3ª lição de Suficiência em Cânones, 07.03.1551:1551-03-07
1 curso de Cânones em 1554:1554-00-00

Graus:
Licenciado em Cânones: 1551-07-02 (02.07.1551)




#### Show as entry (in a dictionary)

In [10]:
print(aluno.as_entry())

[129027] Simão Cabral. Cânones (Ficha original: Dialética) Dialética (Ficha original: Dialética), 1540:1551.
F. Aires Pires Cabral. N. Tomar.
Matr.:  1540-10-25 (Cânones), 1540-10-25 (Dialética).
Ex.:  1551-06-27: Exame Privado  27.06.1551..
G.  Licenciado em Cânones 1551-07-02.


#### Mostrar os attributos como uma tabela

---

#### Show the attributes as a table

In [11]:

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_colwidth',100)

df = group_attributes([id], db=db)
df.info()
df = df[~df['the_type'].str.contains(".", regex = False)]  # remove redundant attributes with "." in the name
df = df[['the_date','the_type','the_value','attr_obs']].sort_values(['the_date','the_type','the_value'])
df.fillna(" ")

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 129027 to 129027
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   the_type   28 non-null     object
 1   the_value  28 non-null     object
 2   the_date   28 non-null     object
 3   attr_obs   28 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


,the_date,the_type,the_value,attr_obs
id,,,,
129027,1540-10-25,faculdade,Cânones,Ficha original: Dialética
129027,1540-10-25,faculdade,Dialética,Ficha original: Dialética
129027,1540-10-25,faculdade-original,Dialética,
129027,1540-10-25,matricula-faculdade,Cânones,25.10.1540
129027,1540-10-25,matricula-faculdade,Dialética,25.10.1540
129027,1540-10-25,naturalidade,Tomar,
129027,1540-10-25,nome,Simão Cabral,
129027,1540-10-25,nome-apelido,Cabral,
129027,1540-10-25,nome-geografico,Tomar,
